# Reproduction - Baseline

> Chung, J.S., Huh, J., Mun, S., Lee, M., Heo, H.S., Choe, S., Ham, C., Jung, S., Lee, B.-J., Han, I., 2020. In defence of metric learning for speaker recognition. arXiv Prepr. arXiv2003.11982.
> 由官方提供

## 建模

### 数据准备

####  数据读取

- 输入波形的帧长度：训练过程 200 帧，评估过程 300 帧，每帧 160 个采样点，16 kHz 采样率。
- 训练的批量大小：200
- 每个 epoch 中的单个说话人的最大段数量：100
- 数据读取的多线程数量：5

#### 训练数据和测试数据

- 训练集列表："/workspace/rwang/voxceleb/train_list.txt"
- 测试集列表："/workspace/rwang/VoxSRC2020/data/verif/trials.txt"
- 训练集所在的根目录："/workspace/rwang/voxceleb/voxceleb2/"
- 测试集所在的根目录："/workspace/rwang/voxceleb/"

### 模型准备

#### 模型介绍

- 输入：40 维度梅尔滤波器组
- 模型结构：ResNet34L
- 编码器/池化层：Self-Attention Pooling (SAP)
- 最后全链接层的嵌入大小：512

#### 读取与保存的路径

- 初始化模型，适用于预训练模型的载入：""
- 模型和日志的保存路径：""

### 训练与测试

#### 训练参数

- 评估间隔的 epoch 数量：10
- epoch 数量：500
- 损失函数：AMSoftmax
- 优化器：Adam

### 损失函数相关参数

- 困难负样本挖掘概率 (PairwiseLoss - triplet/contrastive)：0.5
- 每批困难负样本等级 (PairwiseLoss - triplet/contrastive)：10
- 损失的间隔 (AMSoftmax, AAMsoftmax, PairwiseLoss)：0.3
- 损失的尺度 (AMSoftmax, AAMsoftmax)：30
- 说话人数量 (SoftmaxLoss, AMSoftmax, AAMsoftmax)：5994
- 其它损失函数：AngleProtoLoss, GE2ELoss, ProtoLoss

#### 学习率

- 初始的学习率：0.001
- 每次评估间隔衰减率：0.95

#### 测试许可

- 是否仅评估：False/True

## 数据准备

####  数据读取

- 输入波形的帧长度：训练过程 200 帧，评估过程 300 帧，每帧 160 个采样点，16 kHz 采样率。
- 训练的批量大小：200
- 每个 epoch 中的单个说话人的最大段数量：100
- 数据读取的多线程数量：5

#### 训练数据和测试数据

- 训练集列表："/workspace/rwang/voxceleb/train_list.txt"
- 测试集列表："/workspace/rwang/VoxSRC2020/data/verif/trials.txt"
- 训练集所在的根目录："/workspace/rwang/voxceleb/voxceleb2/"
- 测试集所在的根目录："/workspace/rwang/voxceleb/"

In [2]:
from base import *

In [3]:
trainlst = "/workspace/rwang/competition/voxsrc2020/train_part.txt"
testlst = "/workspace/rwang/VoxSRC2020/data/verif/trials.txt"
traindir = "/workspace/rwang/voxceleb/voxceleb2/"
testdir = "/workspace/rwang/voxceleb/"
maptrain5994 = "/workspace/rwang/competition/voxsrc2020/maptrain5994.txt"

batch_size = 512
num_worker = 8
max_utt_per_spk = 20
L = 32240
num_worker = 6

In [4]:
trainloader = load_train(trainlst, traindir, maptrain5994, L=L, batch_size=batch_size, num_worker=num_worker, max_utt_per_spk=max_utt_per_spk)

In [5]:
trials_loader = load_trial(testlst, testdir, batch_size=batch_size, num_worker=num_worker)

## 模型准备

#### 模型介绍

- 输入：40 维度梅尔滤波器组
- 模型结构：ResNet34L
- 编码器/池化层：Self-Attention Pooling (SAP)
- 最后全链接层的嵌入大小：512

#### 读取与保存的路径

- 初始化模型，适用于预训练模型的载入：""
- 模型和日志的保存路径：""

In [6]:
n_mel = 40
n_out = 512

## 训练与测试

#### 训练参数

- 评估间隔的 epoch 数量：10
- epoch 数量：500
- 损失函数：AMSoftmax
- 优化器：Adam

### 损失函数相关参数

- 损失的间隔 (AMSoftmax, AAMsoftmax, PairwiseLoss)：0.3
- 损失的尺度 (AMSoftmax, AAMsoftmax)：30
- 说话人数量 (SoftmaxLoss, AMSoftmax, AAMsoftmax)：5994

#### 学习率

- 初始的学习率：0.001
- 每次评估间隔衰减率：0.95

In [7]:
eval_interval = 20
num_epoch = 500
margin = 0.2
scale = 30
num_speaker = 5994
lr = 0.001
gamma = 0.95
log_dir = "/workspace/rwang/competition/voxsrc2020/logs/exp-24"
device = "cuda:1"

In [8]:
eval_L = 48240
mode = "norm2"
num_eval = 10
eval_batch_size = 30
embed_norm = True

## 模型训练

In [9]:
net = ResNetSE34L(nOut=n_out)
top = AMSoftmax(in_feats=n_out, n_classes=num_speaker, m=margin, s=scale)
snet = SpeakerNet(device=device, log_dir=log_dir, eval_interval=eval_interval, lr=lr, gamma=gamma, top=top, net=net)

Embedding size is 512, encoder SAP.
Initialized AMSoftmax margin = 0.200 scale = 30.000


```cmd
python ./trainSpeakerNet.py --model ResNetSE34L --encoder SAP --trainfunc amsoftmax --save_path data/exp1 --nSpeakers 5994 --batch_size 200 --scale 30 --margin 0.3 --train_list /workspace/rwang/voxceleb/train_list.txt --test_list /workspace/rwang/voxceleb/test_list.txt --train_path /workspace/rwang/voxceleb/voxceleb2 --test_path /workspace/rwang/voxceleb/voxceleb1
```
- （官方）训练速率：800 样本/秒

In [ ]:
eer, thresh, all_scores, all_labels, all_trials, trials_feat = snet.evaluate(trials_loader, 0)

 30%|███       | 1417/4673 [03:13<07:26,  7.30it/s]

In [ ]:
snet.train(trainloader, trials_loader, num_epoch=num_epoch, step_num=11, 
           mode=mode, eval_L=eval_L, num_eval=num_eval, eval_batch_size=eval_batch_size, 
           embed_norm=embed_norm)